In [1]:
from petrelic.multiplicative.pairing import G1, G2, GT
from credential import *
import pickle
import hashlib

In [2]:
L = 5
sk, pk = generate_key([b"0"] * L)

user_attributes = [Attribute(0, "key0", b'0'), Attribute(1, "key1", b'1'), Attribute(2, "key2", b'2')]
issuer_attributes = [Attribute(3, "key3", b'3'), Attribute(4, "key4", b'4')]

In [3]:
user_attributes[0].to_formatted_string()

"key0:b'0'"

In [4]:
# pick random t from integers modulo p
t = G1.order().random()

# compute commitment
C = pk.g ** t
for attr in user_attributes:
    C *= pk.Y[attr.index] ** bytes_to_Z_p(attr.to_bytes())

In [5]:
C

G1Element(030f2dde9ae3cb7b86eac290fe6738f8aaaab8f7498070979db287a679dfaa4da8bf1c21da26dbe89a5c6d1d7e67f6da0e)

In [6]:
C = pk.g ** t
generators = [pk.g]
prover_inputs = [t]
for attr in user_attributes:
    prover_input = bytes_to_Z_p(attr.to_bytes()) # TODO: check if it makes sense to convert the whole object to bytes or just the value?
    generator = pk.Y[attr.index]
    C *= generator ** prover_input

    generators.append(generator)
    prover_inputs.append(prover_input)

In [7]:
generators

[G1Element(0217f1d3a73197d7942695638c4fa9ac0fc3688c4f9774b905a14e3a3f171bac586c55e83ff97a1aeffb3af00adb22c6bb),
 G1Element(0305728171b162ae54f89d61bfc2d7648ffbd6a04f3b423534c8f4ff6584bfd2be3c9f3561542ce32de05c7d9852439c2c),
 G1Element(0217c7a888127e9bb77d2cb8895ed0a996b58091934fae64a50e57e01b964dda43f5665fe4a61db757eecdc579089824a0),
 G1Element(0301fadfd92d497f40200b36ce346b6357be62ae4be8f1667a8869371e1bf25ba64739ac81b1531f075766ec06b6daf7f5)]

In [8]:
prover_inputs

[Bn(31334862284653101195464593747336400524771700998045460668997937692749282587976),
 Bn(28812505291598470597331543819607234694483952937860267074902597977240210130406),
 Bn(7823716847220878955081924320961187979837427800045809367666488457301086238516),
 Bn(26883726244086053348831521628195297589822494354156572025663789223720282439552)]

In [9]:
randoms, R = get_zkp_commitment(generators)
print("Randoms: ", randoms)
print("R: ", R)

Randoms:  [Bn(37971201155163789907927021642123064697828018714853460936071512652698350671072), Bn(13143152699880265861603266196391555147505825211061159076311240171530316185220), Bn(50918582751319172171700372004414259124183407831010481055880650434555323358637), Bn(31316796374118732790449558602540145072705919422913198551703794595834920443907)]
R:  G1Element(0204fd781a460f992ccec637a6aec3f5673ee921f4f99a28db63cb9f289d65f188bdbcd83ac1e4bc7e4d42ec5e3f33c7b2)


In [10]:
c = get_zkp_challenge(generators, C, R)
print("Challenge: ", c)

Challenge:  3759759404816717415287708258211741501721078259879623955332799162780114740276


In [11]:
c

Bn(3759759404816717415287708258211741501721078259879623955332799162780114740276)

In [12]:
s = get_zkp_response(randoms, c, prover_inputs)
print("Response: ", s)
# response[3] = Bn(2)

Response:  [Bn(38676923015151014816991151974089026008858060119838584246459977019189957915976), Bn(33261141683883623339798280963996090567399405400496735371945799395029378054999), Bn(40792280185256034988488958274513031211305962598105557844651052767303201892631), Bn(9942044057242031538819764297844134309292243362885272703074869542739558386532)]


In [13]:
verify_zkp(C, generators, c, s)

True

# SHOWING

In [14]:
def create_disclosure_proof(
        pk: PublicKey,
        credential: AnonymousCredential,
        hidden_attributes: List[Attribute],
        message: bytes
    ) -> DisclosureProof:
    """ Create a disclosure proof """
    rdm_part = credential.sigma_1.pair(pk.g_tilde) ** credential.t
    for h_id in range(len(hidden_attributes)):
        idx = hidden_attributes[h_id].index
        rdm_part *= credential.sigma_1.pair(pk.Y_tilde[idx]) ** bytes_to_Z_p(hidden_attributes[h_id].to_bytes())

    return DisclosureProof(rdm_part, credential)


def verify_disclosure_proof(
        pk: PublicKey,
        disclosure_proof: DisclosureProof,
        message: bytes,
        # todo: check how can be retrieved otherwise?
        disclosed_attributes: List[Attribute]
    ) -> bool:
    """ Verify the disclosure proof

    Hint: The verifier may also want to retrieve the disclosed attributes
    """
    credential = disclosure_proof.credential_showed
    denominator = credential.sigma_1.pair(pk.X_tilde)

    numerator = credential.sigma_2.pair(pk.g_tilde)
    for id in range(len(disclosed_attributes)):
        idx = disclosed_attributes[id].index
        numerator *= credential.sigma_1.pair(pk.Y_tilde[idx]) ** (
                bytes_to_Z_p(disclosed_attributes[id].to_bytes()) * -1)

    return numerator / denominator == disclosure_proof.proof and credential.sigma_1 != G1.unity()

In [15]:
# petrelic pairing

g = G1.generator()
element = G2.hash_to_point(os.urandom(32))

In [16]:
g

G1Element(0217f1d3a73197d7942695638c4fa9ac0fc3688c4f9774b905a14e3a3f171bac586c55e83ff97a1aeffb3af00adb22c6bb)

In [17]:
element

G2Element(030cae009485797c4424f458cbe179f89a587236425e101e90775c2ef1e5709e76e543dabe02618d0c02720a4a0f51133717b9b5ac6cb9cbe095b02dedd7dd525056a8b28849255f1c41eacd722710f8768769344749bcdea8c4422cd0885529ae)

In [18]:
g.pair(element)

GTElement(00e275a3c7f81ecbb9a41738afe3f5c6e13cb92fab7cb1784c2ae91bc1827d0d7d1c692a0dfd328ece3a306c1eb3c87f059810653de9e4a9a38e0f31b7d4abea806d52311b13e9ebe045215aaa0dfbc3dfe6fee6b9e477f205d9907222e26052191eb7cc32e6f9463230b58fc4e685274e05b2517b3f16aa876d23e40e99996a496c388a575286e734e53c1323441d890c6ba48d232064602dad0ba84c8a6c6008b9f1fbfa8efa0712016f1a52b31c071f2ed8f099d89970f34432bb46b766170f77b3fe3a2f2f33b25d0545240e0f24ed344373b627d874f1fa030bdbcc226217f55cebdd10d1021f3965bfcc80492017990b8e3d5daf092b3975ed45e143f47b3105df2dfef69ceeca314edec5c5e3f5741c46c755f1c9f367fdd9943e8e6619946351f577d960b8344413302277eae8742412019af248ef6991e06c7d91e7b2eeb43991d33e8f620c823ef45418a608060ad98c09e73583245fba0b2b14da7da285274704abb8624a772fc15c77bb837777bb73a654d6d218f41d01c079ec)

### Create disclosure proof

In [32]:
def create_disclosure_proof_1(
        pk: PublicKey,
        credential: AnonymousCredential,
        hidden_attributes: List[Attribute],
        message: bytes
    ) -> Tuple[DisclosureProof, Any]:
    """ Create a disclosure proof """
    
    # compute Pedersen commitment (RHS)
    C = credential.sigma_1.pair(pk.g_tilde) ** credential.t
    generators = [credential.sigma_1.pair(pk.g_tilde)]
    prover_inputs = [credential.t]

    for hidden_attr in hidden_attributes:
        idx = hidden_attr.index
        generator = credential.sigma_1.pair(pk.Y_tilde[idx])
        prover_input = bytes_to_Z_p(hidden_attr.to_bytes())
        C *= generator ** prover_input

        generators.append(generator)
        prover_inputs.append(prover_input)
    
    # compute a non-interactive proof pi
    c, s = generate_zkp(generators, prover_inputs, C)
    pi = ZKProof(generators, c, s)

    return DisclosureProof(pi, credential), C

### Verify

In [33]:
def verify_disclosure_proof_1(
        pk: PublicKey,
        disclosure_proof: DisclosureProof,
        message: bytes,
        # todo: check how can be retrieved otherwise?
        disclosed_attributes: List[Attribute]
    ) -> Any:
    """ Verify the disclosure proof

    Hint: The verifier may also want to retrieve the disclosed attributes
    """
    # disclosure proof
    credential = disclosure_proof.credential_showed
    pi = disclosure_proof.pi

    # compute Pedersen commitment (LHS)
    # the user does not send the Pedersen commitment, the verifier can compute it from the disclosed attributes
    # numerator
    numerator = credential.sigma_2.pair(pk.g_tilde)
    for disclosed_attr in disclosed_attributes:
        idx = disclosed_attr.index
        numerator *= credential.sigma_1.pair(pk.Y_tilde[idx]) ** (bytes_to_Z_p(disclosed_attr.to_bytes()).int_neg())
    # denominator
    denominator = credential.sigma_1.pair(pk.X_tilde)
    # Pedersen commitment
    C = numerator / denominator

    return C

    # verify ZKP
    # return credential.sigma_1 != G1.unity() and verify_zkp(C, pi.generators, pi.c, pi.s)

In [34]:
L = 5
sk, pk = generate_key([b"0"] * L)
user_attributes = [Attribute(0, "key0", b'0'), Attribute(1, "key1", b'1'), Attribute(2, "key2", b'2')]
issuer_attributes = [Attribute(3, "key3", b'3'), Attribute(4, "key4", b'4')]

In [35]:
issue_request, t = create_issue_request(pk, user_attributes)

In [36]:
blind_signature = sign_issue_request(sk, pk, issue_request, issuer_attributes)

In [37]:
credential = obtain_credential(pk, blind_signature, t)

In [38]:
user_attributes_bytes = [attr.to_bytes() for attr in user_attributes]
issuer_attributes_bytes = [attr.to_bytes() for attr in issuer_attributes]
attributes = user_attributes_bytes + issuer_attributes_bytes
verify(pk, credential, attributes)

True

In [39]:
hidden_attributes = [Attribute(0, "key0", b'0'), Attribute(1, "key1", b'1'), Attribute(2, "key2", b'2'), Attribute(3, "key3", b'3')]
disclosed_attributes = [Attribute(4, "key4", b'4')]

In [40]:
anon_credential = credential.anonymize()
anon_credential.sigma_1

G1Element(020c6b12ac3486a456fcf5da8326ff22855d929355154a8f274a27467c9fdcfb5f047abc6f74d92f90a80e544b58c5f1c2)

In [41]:
anon_credential.sigma_2

G1Element(0318176da52d93cd8c24230aec0f632dff68480c9f19148c1d9192939b385985f663762f5aa574aba39866eed01d0b344c)

In [42]:
disclosure_proof, C = create_disclosure_proof_1(pk, anon_credential, hidden_attributes, b"message")

In [43]:
C

GTElement(17bbf5b342df074c54a2b6846f8e9604da389f5fd5a32f5717714a8d02aa229da04eb4a1ee9f76ed8ebb35296b9a039d0e1a801e32ac3379fef1e713c30e6a4fbacbc7d3272d1c3f5bccaf08f9795b8ccf54d8fae5d8d6fb1dd988cfd6330343119709152030c75b949a906b6b5671a981caf39a2c4cbac74fe06d01c107fe83765b41d052d2276fbbc7bd553020424f0c096ea11cb8a07391bcc5e7df12bff21ae2357ba16b1c645b3fdb55494b02608b4327372ef29912b0b657d14120fe2b0055e8023c0c89a97d77ae19991cfe749d131d0426914b905e03ccb2190b051851ea15c6865513485d80fc6cb43fddce05c81998d512e1c13a5f42031cffe905f3b523aa5575da8cc8e645f32379ab3ad6c9671b39cc6945c2adc2791137293914f0d5c8a90f99640f2a75d1075b9dde4c1021419f8676892107615bff2464cc2480a4c0d300818294393d9211c0fb1710a45923d22d8f7742331d9d5419664c5ab766e1046f0306b0edc3cc51e6e6658bc95eaf0dc6ffe4ed5a29022070d95f)

In [44]:
verify_disclosure_proof_1(pk, disclosure_proof, b"message", disclosed_attributes)

GTElement(0a7eab2160e27b4dcfe7ddb9f4d39a8b9a7cc28aa05fbec41fb1b805e01be01cee91487e12c049474d72a231ed6ac4a5054bb322494c787a2df93a4a2d609dcb4daa56d0133e89519ed2247636ae7a41a1ac6cf554f706cb9b1e00c339bb408418373f976478062b112d7776510e3c311516adc4552d712856f35fe44d2f832dc621fc2fde189635881392cc6a6d0c5a060b7fa09c609762f161011d1611abcefb4dfd9997b1464b600dda6181638f8d851b7bdb6b4f5d08488381ff7285731a052deef40c00c722f725ea6afc825631368ccd4628b0a3640e882c837788c42de0271707c93d3ec4acf342a7d92a542e0b12681d8c6e5b447e8ad815ebba8f268ded3cfe8546c478a983738336e4ff10c74ee1484f0fabe03016bbd9d0e91d8d17c11f3283f9ce0c94cc8487fc8d7f10fc0501f96ae85e7af0341fd2204438633b190c74d47aae7cc0a1ad5824c0db420c6149b05abba329217be0fab37e6830de6cbe2e6d3005ca16c62f96ac61aaccf1acd0e17c4136c237fc4aebf30fd002)

In [45]:
# This should be true the commitment should be equal LHS = RHS
C == verify_disclosure_proof_1(pk, disclosure_proof, b"message", disclosed_attributes)

False